In [14]:
import requests 
import json
import time
from pprint import pprint 
import pandas as pd 
import numpy as np


In [15]:
# читаем токен, предварительно записанный в файл token.txt, файл положить рядом с ноутбуком
with open('token.txt', 'r') as file_object:
    token = file_object.read().strip()

In [72]:
 #Параметры sort
    #0 — сортировать по умолчанию (аналогично результатам поиска в полной версии сайта);
    #1 — сортировать по скорости роста;
    #2 — сортировать по отношению дневной посещаемости к количеству пользователей;
    #3 — сортировать по отношению количества лайков к количеству пользователей;
    #4 — сортировать по отношению количества комментариев к количеству пользователей;
    #5 — сортировать по отношению количества записей в обсуждениях к количеству пользователей.
        
# читаем токен, предварительно записанный в файл token.txt, файл положить рядом с ноутбуком
with open('token.txt', 'r') as file_object:
    token = file_object.read().strip()

def groups_search_query(key_word, token, sorting=0, type_group='group', count=500 ):
    ''' используем метод VK API groups.search
    https://dev.vk.com/method/groups.search    
    '''
    URL = 'https://api.vk.com/method/{}' 
    method = 'groups.search'   
    params = {
        'q': key_word,
        'access_token': token,
        'sort': sorting,
        'count': count,
        'type': type_group,
        'v':'5.131'
    }
    res = requests.get(URL.format(method), params).json() # собираем запрос сразу обрабатываем методом .json()
    result = res['response']['items'] 
    return result

key_word = 'авто' # выборка по интересующему нас слову
groups_search_df =pd.DataFrame(groups_search_query(key_word, token, count=500, sorting=2, type_group='group'))

groups_search_df.tail()



,id,name,screen_name,is_closed,type,photo_50,photo_100,photo_200
495,75759238,АВТО-НьЮс,autocentre1autocentre,0,group,https://sun1-87.userapi.com/s/v1/if2/a9e5qnlzh...,https://sun1-87.userapi.com/s/v1/if2/Wku61Ayvl...,https://sun1-87.userapi.com/s/v1/if2/TculGj1Pb...
496,154030241,АВТО,avtocar138,0,group,https://sun1-91.userapi.com/s/v1/if2/BU07P2vqS...,https://sun1-91.userapi.com/s/v1/if2/QvB2leWgd...,https://sun1-91.userapi.com/s/v1/if2/jN3BX1dob...
497,22908167,Ильиногорск авто,club22908167,0,group,https://vk.com/images/community_50.png,https://vk.com/images/community_100.png,https://vk.com/images/community_200.png
498,16058965,Авто Новогрудка.Авто новости.,club16058965,1,group,https://sun9-74.userapi.com/c635/g16058965/c_d...,https://sun9-74.userapi.com/c635/g16058965/b_3...,https://sun9-74.userapi.com/c635/g16058965/b_3...
499,21509735,АВТО,club21509735,0,group,https://sun9-45.userapi.com/c9773/g21509735/c_...,https://sun9-45.userapi.com/c9773/g21509735/b_...,https://sun9-45.userapi.com/c9773/g21509735/b_...


In [73]:
groups_search_df.to_csv('groups_df', sep=',') # сохраняем промежуточные результаты

### Вариант использования прямого метода *groups.getMembers*:

In [ ]:


# df = pd.read_csv('groups_df', sep=',')

# def vk_groups_members(group_id):
#     ''' function for applying vk.com/groups.getMembers
#     https://dev.vk.com/method/groups.getMembers
#     '''
#     time.sleep(0.32)
#     with open('token.txt', 'r') as file:
#         token = file.read().strip()

#     url = 'https://api.vk.com/method/{}' # Указываем адрес обращения
#     method = 'groups.getMembers'
#     params = {
#         'access_token': token,
#         'group_id': group_id,
#         'count': 1,
#         'offset': 0,
#         'v': 5.131
#         } 
    
#     max_count_req = requests.get(url.format(method), params = params).json()
#     try:
#         max_count = max_count_req['response']['count']
#     except:
#         return np.NaN
#     offset = 0
#     res_df = list()
#     count = 500
#     while offset < max_count:
#         # начиная с того места, где закончили на предыдущей итерации (offset) 
#         # print('Выгружаю {} пользователей с offset = {}'.format(count, offset))   
#         params = {
#         'access_token': token,
#         'group_id': group_id,
#         'sort': 2,
#         'count': count,
#         'offset': offset,
#         'v': 5.131
#         } 
#         time.sleep(0.32)
#         data = requests.get(url.format(method), params = params).json()
#         # pprint(data)
#         res_df.extend(data['response']['items'])
#         offset += count
#     return res_df


# df['members'] = df['id'].apply(lambda x: vk_groups_members(x))
# df.to_csv('df_with_members.csv')


### Вариант использования прямого метода *execute groups.getMembers*

In [64]:

df = pd.read_csv('groups_df', sep=',')
def vk_groups_members(group_id):
    ''' function for applying vk.com/groups.getMembers
    https://dev.vk.com/method/groups.getMembers
    '''
    with open('token.txt', 'r') as file: #берем токен из файла
        token = file.read().strip()
   
    time.sleep(0.3)
    url = "https://api.vk.com/method/execute?"
    method = 'groups.getMembers'
    params = {
        'access_token': token,
        'group_id': group_id,
        'count': 1,
        'offset': 0,
        'v': 5.131
        } 
    
    code = 'return [API.{}({})];'.format(method, params)
    data = dict(code=code, access_token=token, v='5.131')
    max_count_req = requests.post(url=url, data=data).json()
    try:
        max_count = max_count_req['response'][0]['count']
        print(f'right way {max_count}')
    except:
        max_count = max_count_req['response']
        print(f'wrong way {max_count}')
        return np.NaN
    
    offset = 0
    resuslt_list = list()
    count = 500
    while offset < max_count:
        # начиная с того места, где закончили на предыдущей итерации (offset) 
        # print('Выгружаю {} пользователей с offset = {}'.format(count, offset))   
        params = {
        'access_token': token,
        'group_id': group_id,
        'sort': 2,
        'count': count,
        'offset': offset,
        'v': 5.131
        }
        code = 'return [API.{}({})];'.format(method, params)
        data = dict(code=code, access_token=token, v='5.131')
        json_to_list = requests.post(url=url, data=data).json()
        # pprint(json_to_list)
        time.sleep(0.2)
        try:
            lst = list(json_to_list['response']['items'])
        except:
            pprint(f'wrong way list {lst}')
            return lst
            # resuslt_list.extend(json_to_list['response'][0]['items'])
        # Увеличиваем смещение на количество строк, которое мы уже выгрузили 
        resuslt_list.extend(lst)
        offset += count
    return resuslt_list


df['members'] = df['id'].apply(lambda x: vk_groups_members(x))
df.to_csv('df_with_members.csv')



KeyError: 'response'

In [56]:
df.to_csv('df_with_members.csv')
df.head()


,Unnamed: 0,id,name,screen_name,is_closed,type,photo_50,photo_100,photo_200
0,489,208977747,Крым Авто,clubcrimeaauto,0,group,https://sun1-84.userapi.com/s/v1/ig2/eCDNxv89H...,https://sun1-84.userapi.com/s/v1/ig2/TDCKI10YU...,https://sun1-84.userapi.com/s/v1/ig2/vvPmZxDBI...
1,143,208387275,Купля продажа авто,club208387275,0,group,https://sun1-26.userapi.com/s/v1/ig2/nmbPFtYRh...,https://sun1-26.userapi.com/s/v1/ig2/viFdPcxsH...,https://sun1-26.userapi.com/s/v1/ig2/_WFDncNsS...
2,79,207721278,Эксгибиционизм в авто,club207721278,0,group,https://sun1-17.userapi.com/s/v1/ig2/-X3g6PTXz...,https://sun1-17.userapi.com/s/v1/ig2/hvVRMsxsu...,https://sun1-17.userapi.com/s/v1/ig2/eNg63kcKF...
3,467,206811903,Авто Аркалык,club206811903,0,group,https://sun1-84.userapi.com/s/v1/ig2/ND5ffjZ-b...,https://sun1-84.userapi.com/s/v1/ig2/kjhO7kqp2...,https://sun1-84.userapi.com/s/v1/ig2/WnLjjj07r...
4,71,205718149,Авто | Волгоград,autovolgograd,0,group,https://sun1-14.userapi.com/s/v1/ig2/8oMX4u26L...,https://sun1-14.userapi.com/s/v1/ig2/ygi-tbp1g...,https://sun1-14.userapi.com/s/v1/ig2/MJDnUFTAk...
